# Code for training CNN on MNIST and saving weights


In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow-gpu==2.4

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#initialization code required to make tensorflow work on my systemabs
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
#disabling eager execution 
tf.compat.v1.disable_eager_execution()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Tensorflow version: ",tf.__version__)

In [38]:
from art.utils import load_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

### I am using a simple covnet from the official ART for keras mnist tutorial linked [here](https://github.com/Trusted-AI/adversarial-robustness-toolbox/blob/main/examples/get_started_keras.py)

ART's load_mnist already loads in preprocess MNIST data (features [0,1] scaled and labels made categorical)

In [39]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

Define and compile the model architecture: taken directly from official ART tutorial linked above

In [40]:
#define
model = Sequential()
model.add(Conv2D(filters=4, kernel_size=(5, 5), strides=1, activation="relu", input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=10, kernel_size=(5, 5), strides=1, activation="relu", input_shape=(23, 23, 4)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))


In [41]:
#compile: Categorical Crossentropy Loss Function and Adam Optimizer used
model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=0.01), metrics=["accuracy"])

Training model: I use more epochs and a larget batch size than the tutorial

In [ ]:
batch_size = 128
epochs = 15
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
